In [3]:
from update_vars import analysis_date_list, SCHED_GCS, GTFS_DATA_DICT

In [4]:
import datetime
import geopandas as gpd
import numpy as np
import pandas as pd
import sys

from loguru import logger

from calitp_data_analysis import utils
from shared_utils import rt_utils
from segment_speed_utils import helpers
from segment_speed_utils.project_vars import PROJECT_CRS
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS

In [5]:
analysis_date_list

['2025-01-15']

In [7]:
analysis_date = analysis_date_list[0]

## Next steps to solve https://github.com/cal-itp/data-analyses/pull/1356

In [2]:
import stop_times_with_direction as sd

In [34]:
EXPORT_FILE = GTFS_DATA_DICT.rt_vs_schedule_tables.stop_times_direction

scheduled_stop_times = sd.prep_scheduled_stop_times(analysis_date)

trip_cols = ["trip_instance_key"]
trip_stop_cols = ["trip_instance_key", "stop_sequence"]

In [64]:
scheduled_stop_times.query('trip_instance_key == "0c9b170807546736de0820cf8860bc00"').head(10) #  not sorted...

,feed_key,stop_id,stop_sequence,schedule_gtfs_dataset_key,trip_instance_key,shape_array_key,stop_name,geometry
2714578,7a5a0fa93eb3c7a402c0887a4f11f066,9255,37,0666caf3ec1ecc96b74f4477ee4bc939,0c9b170807546736de0820cf8860bc00,59459dbb9f7ce418d6f88f65e905d583,Chase / Katherine,POINT (143436.362 -420025.038)
2714814,7a5a0fa93eb3c7a402c0887a4f11f066,9256,38,0666caf3ec1ecc96b74f4477ee4bc939,0c9b170807546736de0820cf8860bc00,59459dbb9f7ce418d6f88f65e905d583,Chase / Lennox,POINT (143169.561 -420048.613)
2716121,7a5a0fa93eb3c7a402c0887a4f11f066,695,35,0666caf3ec1ecc96b74f4477ee4bc939,0c9b170807546736de0820cf8860bc00,59459dbb9f7ce418d6f88f65e905d583,Chase / Ranchito,POINT (143956.624 -419845.037)
2716157,7a5a0fa93eb3c7a402c0887a4f11f066,9254,36,0666caf3ec1ecc96b74f4477ee4bc939,0c9b170807546736de0820cf8860bc00,59459dbb9f7ce418d6f88f65e905d583,Chase / Hazeltine,POINT (143668.058 -419967.744)
2716193,7a5a0fa93eb3c7a402c0887a4f11f066,696,39,0666caf3ec1ecc96b74f4477ee4bc939,0c9b170807546736de0820cf8860bc00,59459dbb9f7ce418d6f88f65e905d583,Van Nuys / Chase,POINT (142883.028 -420093.918)
2742567,7a5a0fa93eb3c7a402c0887a4f11f066,6744,114,0666caf3ec1ecc96b74f4477ee4bc939,0c9b170807546736de0820cf8860bc00,59459dbb9f7ce418d6f88f65e905d583,Valley Circle / Canzonet,POINT (125364.090 -426918.777)
2742581,7a5a0fa93eb3c7a402c0887a4f11f066,14487,86,0666caf3ec1ecc96b74f4477ee4bc939,0c9b170807546736de0820cf8860bc00,59459dbb9f7ce418d6f88f65e905d583,Saticoy / Sale,POINT (127206.779 -422123.347)
2742598,7a5a0fa93eb3c7a402c0887a4f11f066,15184,98,0666caf3ec1ecc96b74f4477ee4bc939,0c9b170807546736de0820cf8860bc00,59459dbb9f7ce418d6f88f65e905d583,Vanowen / Berquist,POINT (125847.611 -423758.251)
2742612,7a5a0fa93eb3c7a402c0887a4f11f066,1346,79,0666caf3ec1ecc96b74f4477ee4bc939,0c9b170807546736de0820cf8860bc00,59459dbb9f7ce418d6f88f65e905d583,Saticoy / De Soto,POINT (130023.517 -422076.878)
2742629,7a5a0fa93eb3c7a402c0887a4f11f066,15106,106,0666caf3ec1ecc96b74f4477ee4bc939,0c9b170807546736de0820cf8860bc00,59459dbb9f7ce418d6f88f65e905d583,Valley Circle / Hamlin,POINT (123902.277 -424437.038)


In [15]:
trip_cols = ["trip_instance_key"]
trip_stop_cols = ["trip_instance_key", "stop_sequence"]

In [28]:
stop_meters = sd.get_projected_stop_meters(scheduled_stop_times, analysis_date)

/opt/conda/lib/python3.9/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)


In [29]:
gdf = scheduled_stop_times[
    trip_stop_cols + ["stop_id", "stop_name", "geometry"]
].assign(
    stop_meters = stop_meters
)

In [30]:
gdf.query('trip_instance_key == "0c9b170807546736de0820cf8860bc00"')

,trip_instance_key,stop_sequence,stop_id,stop_name,geometry,stop_meters
2714578,0c9b170807546736de0820cf8860bc00,37,9255,Chase / Katherine,POINT (143436.362 -420025.038),20462.186863
2714814,0c9b170807546736de0820cf8860bc00,38,9256,Chase / Lennox,POINT (143169.561 -420048.613),12921.438656
2716121,0c9b170807546736de0820cf8860bc00,35,695,Chase / Ranchito,POINT (143956.624 -419845.037),1247.844911
2716157,0c9b170807546736de0820cf8860bc00,36,9254,Chase / Hazeltine,POINT (143668.058 -419967.744),5083.356481
2716193,0c9b170807546736de0820cf8860bc00,39,696,Van Nuys / Chase,POINT (142883.028 -420093.918),18517.731848
...,...,...,...,...,...,...
2746726,0c9b170807546736de0820cf8860bc00,1,30021,Burbank Airport Ritc,POINT (151920.171 -423388.371),36133.141573
2746822,0c9b170807546736de0820cf8860bc00,118,3325,Mulholland / Willens,POINT (126618.430 -428461.805),18103.141413
2746836,0c9b170807546736de0820cf8860bc00,130,6575,Topanga Canyon / Erwin,POINT (128506.351 -424953.278),18103.141413
2746897,0c9b170807546736de0820cf8860bc00,43,15124,Van Nuys / Arminta,POINT (142901.964 -421307.549),33286.017759


In [21]:
gdf2 = gdf.sort_values(trip_stop_cols) #  sort here

In [62]:
#  too slow on full gdf to add to script?
check_monotonic = gdf2.iloc[:100000,:].groupby(trip_cols).stop_sequence.apply(lambda x: x.is_monotonic_increasing)
assert check_monotonic.all(), 'gdf must be sorted by trip_instance_key, stop_sequence'

In [25]:
gdf3 = gdf2.assign(
    prior_geometry = (gdf2.groupby(trip_cols)
                      .geometry
                      .shift(1)),
    prior_stop_sequence = (gdf2.groupby(trip_cols)
                           .stop_sequence
                           .shift(1)),
    # add subseq stop info here
    subseq_stop_sequence = (gdf2.groupby(trip_cols)
                            .stop_sequence
                            .shift(-1)),
    subseq_stop_id = (gdf2.groupby(trip_cols)
                      .stop_id
                      .shift(-1)),
    subseq_stop_name = (gdf2.groupby(trip_cols)
                      .stop_name
                      .shift(-1)),
).fillna({
    **{c: "" for c in ["subseq_stop_id", "subseq_stop_name"]}
})

In [ ]:
gdf3.query('trip_instance_key == "0c9b170807546736de0820cf8860bc00"').head(3) #  looks good after sort

In [53]:
import importlib
importlib.reload(sd)

<module 'stop_times_with_direction' from '/home/jovyan/data-analyses/gtfs_funnel/stop_times_with_direction.py'>

In [54]:
test_fix = sd.assemble_stop_times_with_direction(analysis_date, GTFS_DATA_DICT)

/opt/conda/lib/python3.9/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
2025-02-12 17:10:33.733 | INFO     | stop_times_with_direction:assemble_stop_times_with_direction:212 - scheduled stop times with direction 2025-01-15: 0:06:49.325982


In [56]:
EXPORT_FILE = GTFS_DATA_DICT.rt_vs_schedule_tables.stop_times_direction

In [59]:
path = f"{RT_SCHED_GCS}{EXPORT_FILE}_{analysis_date}.parquet"

In [60]:
gpd.read_parquet(path).query('trip_instance_key == "0c9b170807546736de0820cf8860bc00"') #  looks good after sort

,feed_key,stop_id,stop_sequence,schedule_gtfs_dataset_key,trip_instance_key,shape_array_key,stop_name,geometry,stop_meters,prior_stop_sequence,subseq_stop_sequence,stop_primary_direction,stop_pair,stop_pair_name
240254,7a5a0fa93eb3c7a402c0887a4f11f066,30021,1,0666caf3ec1ecc96b74f4477ee4bc939,0c9b170807546736de0820cf8860bc00,59459dbb9f7ce418d6f88f65e905d583,Burbank Airport Ritc,POINT (151920.171 -423388.371),36133.141573,NaN,2.0,Unknown,30021__3625,Burbank Airport Ritc__Hollywood Way / Thornton
240255,7a5a0fa93eb3c7a402c0887a4f11f066,3625,2,0666caf3ec1ecc96b74f4477ee4bc939,0c9b170807546736de0820cf8860bc00,59459dbb9f7ce418d6f88f65e905d583,Hollywood Way / Thornton,POINT (152164.852 -423047.563),35377.718010,1.0,3.0,Northbound,3625__12116,Hollywood Way / Thornton__Hollywood Way / Winona
240256,7a5a0fa93eb3c7a402c0887a4f11f066,12116,3,0666caf3ec1ecc96b74f4477ee4bc939,0c9b170807546736de0820cf8860bc00,59459dbb9f7ce418d6f88f65e905d583,Hollywood Way / Winona,POINT (152163.927 -422698.157),17547.436901,2.0,4.0,Northbound,12116__2499,Hollywood Way / Winona__Hollywood Way / Tulare
240257,7a5a0fa93eb3c7a402c0887a4f11f066,2499,4,0666caf3ec1ecc96b74f4477ee4bc939,0c9b170807546736de0820cf8860bc00,59459dbb9f7ce418d6f88f65e905d583,Hollywood Way / Tulare,POINT (152163.038 -422334.989),5248.682609,3.0,5.0,Northbound,2499__3567,Hollywood Way / Tulare__Burbank Airport North
240258,7a5a0fa93eb3c7a402c0887a4f11f066,3567,5,0666caf3ec1ecc96b74f4477ee4bc939,0c9b170807546736de0820cf8860bc00,59459dbb9f7ce418d6f88f65e905d583,Burbank Airport North,POINT (152086.762 -422003.426),6869.157595,4.0,6.0,Northbound,3567__3252,Burbank Airport North__San Fernando / Lockheed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240383,7a5a0fa93eb3c7a402c0887a4f11f066,6575,130,0666caf3ec1ecc96b74f4477ee4bc939,0c9b170807546736de0820cf8860bc00,59459dbb9f7ce418d6f88f65e905d583,Topanga Canyon / Erwin,POINT (128506.351 -424953.278),18103.141413,129.0,131.0,Northbound,6575__6597,Topanga Canyon / Erwin__Victory / Topanga Canyon
240384,7a5a0fa93eb3c7a402c0887a4f11f066,6597,131,0666caf3ec1ecc96b74f4477ee4bc939,0c9b170807546736de0820cf8860bc00,59459dbb9f7ce418d6f88f65e905d583,Victory / Topanga Canyon,POINT (128552.060 -424482.483),13357.937722,130.0,132.0,Northbound,6597__7148,Victory / Topanga Canyon__Victory / Owensmouth
240385,7a5a0fa93eb3c7a402c0887a4f11f066,7148,132,0666caf3ec1ecc96b74f4477ee4bc939,0c9b170807546736de0820cf8860bc00,59459dbb9f7ce418d6f88f65e905d583,Victory / Owensmouth,POINT (128919.099 -424394.634),8608.096062,131.0,133.0,Eastbound,7148__5507,Victory / Owensmouth__Canoga Station
240386,7a5a0fa93eb3c7a402c0887a4f11f066,5507,133,0666caf3ec1ecc96b74f4477ee4bc939,0c9b170807546736de0820cf8860bc00,59459dbb9f7ce418d6f88f65e905d583,Canoga Station,POINT (129268.917 -424046.607),22395.340227,132.0,134.0,Eastbound,5507__5505,Canoga Station__Rocketdyne Layover
